In [1]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score,train_test_split,RandomizedSearchCV

In [2]:
X = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')
X_test.shape

(1459, 80)

In [3]:
X = X.drop('Id', axis = 1)
Id = X_test['Id']
X_test = X_test.drop('Id', axis = 1)
X.shape

(1460, 80)

In [4]:
y = X['SalePrice']
X = X.drop('SalePrice', axis = 1)

In [5]:
X.shape , y.shape, X_test.shape

((1460, 79), (1460,), (1459, 79))

In [6]:
#Removing the features that have more than 1000 null values like PoolQC, Miscfeature
X.dropna(thresh = 1000, inplace = True, axis = 'columns')
X_test.dropna(thresh = 1000 , inplace = True, axis = 'columns')
X_test.shape

(1459, 74)

In [7]:
#Filling the median value at places where there is Nan
X.fillna(X.median() , inplace = True)
X_test.fillna(X.median() , inplace = True)
np.sum(X.isnull().max())

11

In [8]:
# we still have some missing values
X.fillna(value = "Missing", inplace = True)
X_test.fillna(value = "Missing", inplace = True)
np.sum(X.isnull().max())

0

In [9]:
#Label Encoding for categorical features
le = preprocessing.LabelEncoder()
X = X.apply(le.fit_transform)
X_test = X_test.apply(le.fit_transform)
X.head(20)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,5,3,36,327,1,3,3,0,4,0,...,49,0,0,0,0,0,1,2,8,4
1,0,3,51,498,1,3,3,0,2,0,...,0,0,0,0,0,0,4,1,8,4
2,5,3,39,702,1,0,3,0,4,0,...,30,0,0,0,0,0,8,2,8,4
3,6,3,31,489,1,0,3,0,0,0,...,24,108,0,0,0,0,1,0,8,0
4,5,3,55,925,1,0,3,0,2,0,...,70,0,0,0,0,0,11,2,8,4
5,4,3,56,915,1,0,3,0,4,0,...,20,0,17,0,0,10,9,3,8,4
6,0,3,46,552,1,3,3,0,4,0,...,45,0,0,0,0,0,7,1,8,4
7,5,3,40,593,1,0,3,0,0,0,...,156,94,0,0,0,2,10,3,8,4
8,4,4,22,138,1,3,3,0,4,0,...,0,83,0,0,0,0,3,2,8,0
9,14,3,21,222,1,3,3,0,0,0,...,1,0,0,0,0,0,0,2,8,4


In [10]:
#log transformation
y = np.log1p(y)

### XGBoost and RandomizedCV

In [11]:
from xgboost import XGBRegressor

In [12]:
xgboost = XGBRegressor(random_state = 45, n_estimators = 6000)
params = {'learning_rate': np.logspace(-1,-5), 'subsample': [0.1,0.2,0.4,0.5,0.6],'max_depth': range(2,11),'min_child_weight': range(2,10)}


In [13]:
randomized= RandomizedSearchCV(xgboost, params,verbose = True, random_state = 45)
randomized.fit(X,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 13.8min finished


RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=6000, n_job...
       2.44205309e-04, 2.02358965e-04, 1.67683294e-04, 1.38949549e-04,
       1.15139540e-04, 9.54095476e-05, 7.90604321e-05, 6.55128557e-05,
       5.42867544e-05, 4.49843267e-05, 3.72759372e-05, 3.08884360e-0

In [14]:
randomized.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.004941713361323838, max_delta_step=0, max_depth=8,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=6000, n_jobs=0, num_parallel_tree=1, random_state=45,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.4,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
p = randomized.predict(X_test)
p = np.expm1(p)
submit= pd.DataFrame()
submit['Id'] = Id 
submit['SalePrice'] = p
submit.to_csv('submission_XGBoostTrees(log)1.csv', index = False)
submit.head(20)

,Id,SalePrice
0,1461,127534.523438
1,1462,157116.812500
2,1463,175626.796875
3,1464,184086.781250
4,1465,177544.421875
5,1466,164778.015625
6,1467,171150.500000
7,1468,163128.453125
8,1469,173748.609375
9,1470,122575.046875
